# Introduction

**Main Topic**

This notebook is for **Visualizing Impacting Moment** with blind and config speed from `train` *.mp4.

You can analysis **Impact Moment** from the video.

Main source code is from References, I just a little bit customize the module.

**References**

**Sam Huddleston** : [NFL 1st and Future Getting Started](https://www.kaggle.com/samhuddleston/nfl-1st-and-future-getting-started)

# Set up environment

In [ ]:
import imageio
from PIL import Image
import cv2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import subprocess

import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
plt.rcParams['figure.dpi'] = 150

import seaborn as sns

from IPython.display import Video, display

#block those warnings from pandas about setting values on a slice
import warnings
warnings.filterwarnings('ignore')


# Import video_labels

In [ ]:
# Read in the video labels file
video_labels = pd.read_csv('../input/nfl-impact-detection/train_labels.csv')
video_labels.head()

In [ ]:
# Define the video we'll process
video_name = video_labels['video'][0]
video_name

In [ ]:
## Show original video

In [ ]:
# Define the path and then display the video using 
video_path = f"../input/nfl-impact-detection/train/{video_name}"
display(Video(data=video_path, embed=True))

## Define annotate_video

I just added blind & speed parameters from Original NB [NFL 1st and Future Getting Started](https://www.kaggle.com/samhuddleston/nfl-1st-and-future-getting-started)

In [ ]:
# Create a function to annotate the video at the provided path using labels from the provided dataframe, return the path of the video
def annotate_video(video_path: str, video_labels: pd.DataFrame, blind=False, speed=1) -> str:
    VIDEO_CODEC = "MP4V"
    HELMET_COLOR = (255, 255, 255)    # Black
    IMPACT_COLOR = (0, 0, 255)  # Red
    video_name = os.path.basename(video_path)
    
    vidcap = cv2.VideoCapture(video_path)
    width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    output_path = "labeled_" + video_name
    tmp_output_path = "tmp_" + output_path
    fps = 60 * speed
    output_video = cv2.VideoWriter(tmp_output_path, cv2.VideoWriter_fourcc(*VIDEO_CODEC), fps, (width, height))
    frame = 0
    while True:
        it_worked, img = vidcap.read()
        if not it_worked:
            break
        if blind:
            img = img * 0
        
        # We need to add 1 to the frame count to match the label frame index that starts at 1
        frame += 1
        
        # Let's add a frame index to the video so we can track where we are
        img_name = f"{video_name}_frame{frame}"
        cv2.putText(img, img_name, (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, HELMET_COLOR, thickness=2)
    
        # Now, add the boxes
        boxes = video_labels.query("video == @video_name and frame == @frame")
        for box in boxes.itertuples(index=False):
            if box.impact == 1 and box.confidence > 1 and box.visibility > 0:    # Filter for definitive head impacts and turn labels red
                color, thickness = IMPACT_COLOR, 2
            else:
                color, thickness = HELMET_COLOR, 1
            # Add a box around the helmet
            cv2.rectangle(img, (box.left, box.top), (box.left + box.width, box.top + box.height), color, thickness=thickness)
            cv2.putText(img, box.label, (box.left, max(0, box.top - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, thickness=1)
        output_video.write(img)
    output_video.release()
    
    # Not all browsers support the codec, we will re-load the file at tmp_output_path and convert to a codec that is more broadly readable using ffmpeg
    if os.path.exists(output_path):
        os.remove(output_path)
    subprocess.run(["ffmpeg", "-i", tmp_output_path, "-crf", "18", "-preset", "veryfast", "-vcodec", "libx264", output_path])
    os.remove(tmp_output_path)
    
    return output_path

## Set Speed = 0.5 

In [ ]:
# Label the video and display it - this will take a bit
labeled_video = annotate_video(f"../input/nfl-impact-detection/train/{video_name}", video_labels, speed=0.5)
display(Video(data=labeled_video, embed=True))

## Set Blind Video, Speed = 0.3

In [ ]:
# Label the video and display it - this will take a bit
labeled_video = annotate_video(f"../input/nfl-impact-detection/train/{video_name}", video_labels, blind=True, speed=0.3)
display(Video(data=labeled_video, embed=True))